In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url="https://www.flipkart.com/search?q=lenovo%20legion&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"

In [10]:
op = webdriver.ChromeOptions()
#user agent
op.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
+"AppleWebKit/537.36 (KHTML, like Gecko)"
+"Chrome/87.0.4280.141 Safari/537.36")
browser=webdriver.Chrome(options=op)
browser.maximize_window()
browser.get(url)
#wait 5 sec for browser to open
time.sleep(5)

html_content=browser.page_source

soup=BeautifulSoup(html_content,"html.parser")

In [5]:
def switch_tab(product,tab_index):
    link=product.get("href")
    browser.execute_script(f'window.open("{link}");')
    
    WebDriverWait(browser, 10).until(lambda d: len(d.window_handles) > tab_index)
    browser.switch_to.window(browser.window_handles[tab_index])

    time.sleep(5)
    WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CLASS_NAME,"fonts-loaded")))
    new_content=browser.page_source
    soup=BeautifulSoup(new_content,"html.parser")
    return soup


In [6]:
products=soup.find_all("a",attrs={"rel":"noopener noreferrer"},limit=10)

In [9]:
# To store data
data=[]

(products)

[<a class="_1fQZEK" href="/lenovo-legion-5-pro-intel-core-i7-12th-gen-12700h-32-gb-1-tb-ssd-windows-11-home-8-gb-graphics-nvidia-geforce-rtx-3070-16iah7h-gaming-laptop/p/itm665edfacb3bad?pid=COMGKC6Y2RKGAH3J&amp;lid=LSTCOMGKC6Y2RKGAH3JTN4HYC&amp;marketplace=FLIPKART&amp;q=lenovo+legion&amp;store=4rr%2Ftz1&amp;srno=s_1_1&amp;otracker=search&amp;otracker1=search&amp;iid=de37f9ae-4060-4c92-8f08-746d49a34c84.COMGKC6Y2RKGAH3J.SEARCH&amp;ssid=hxfdzw7n5s0000001707657955599&amp;qH=5dc4906269c2c6e5" rel="noopener noreferrer" target="_blank"><div class="MIXNux"><div class="_2QcLo-"><div><div class="CXW8mj" style="height: 200px; width: 200px;"><img alt="Lenovo Legion 5 Pro Intel Core i7 12th Gen 12700H - (32 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVID..." class="_396cs4" loading="eager" src="https://rukminim2.flixcart.com/image/312/312/xif0q/computer/r/y/x/16arh7h-gaming-laptop-lenovo-original-imagkwgyrggbagsh.jpeg?q=70"/></div></div></div><div class="_3wLduG"><div class="_3PzNI-"><span class=

In [11]:
for product in products:
    product_details={}
    #getting name
    name=product.find("div",class_="_4rR01T")
    name=name.text if name else "Name not available"

    #getting price in SGD
    price=product.find("div",class_="_30jeq3 _1_WHN1")
    price="SGD "+str(round(int(price.text[1:].replace(",",""))*0.016,2))

    #rating
    rating=product.find('div', class_='_3LWZlK')
    rating=rating.text if rating else "Rating not available"

    #number of ratings and number of reviews
    combine_rating_reviews=product.find("span",class_="_2_R_DZ")
    if combine_rating_reviews:
        number_of_ratings=combine_rating_reviews.find_all("span")[1].text 
        number_of_reviews=combine_rating_reviews.find_all("span")[3].text
    else:
        number_of_ratings="Number of ratings not available"
        number_of_reviews="Number of reviews not available"

    product_details["Product name"]=name
    product_details["Product price"]=price
    product_details["Number of ratings"]=number_of_ratings
    product_details["Number of reviews"]=number_of_reviews

    if number_of_reviews!=0 or number_of_reviews!="Number of reviews not available":
        soup=switch_tab(product,1)
        div_element=soup.find('div', class_='col JOpGWq')

        if div_element:

            all_a_tags=div_element.find_all('a')
            review_summary_list = []
            review_details_list = []

            #get last page
            if len(all_a_tags)>10:
                soup=switch_tab(all_a_tags[-1],2)
                review_container=soup.find_all("div",class_="_1AtVbE col-12-12")

                for review in review_container:

                    review_summary=review.find("p",class_="_2-N8zT").text if review.find("p",class_="_2-N8zT") else "Review not available"
                    review_summary_list.append(review_summary)

                    review_detail=review.find("div",class_="t-ZTKy").text if review.find("div",class_="t-ZTKy") else "Review detail not available"
                    review_details_list.append(review_detail)

                    product_details["Review Summary"]=review_summary_list
                    product_details["Review Detail"]=review_details_list

                #close tab 2
                browser.close()
            else:
                review_container=soup.find_all("div",class_="_2c2kV-")
                
                for review in review_container:
                    review_summary=review.find("p",class_="_2-N8zT").text if review.find("p",class_="_2-N8zT") else "Review not available"
                    review_summary_list.append(review_summary)

                    review_detail=review.find("div",class_="t-ZTKy").text if review.find("div",class_="t-ZTKy") else "Review detail not available"
                    review_details_list.append(review_detail)

                    product_details["Review Summary"]=review_summary_list
                    product_details["Review Detail"]=review_details_list

        #close tab 1
        browser.switch_to.window(browser.window_handles[1])
        browser.close()
        
    #go back to original tab
    browser.switch_to.window(browser.window_handles[0])
    data.append(product_details)
    


In [14]:
data

[{'Product name': 'Lenovo Legion 5 Pro Intel Core i7 12th Gen 12700H - (32 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVID...',
  'Product price': 'SGD 2319.84',
  'Number of ratings': 'Number of ratings not available',
  'Number of reviews': 'Number of reviews not available'},
 {'Product name': 'Lenovo Lenovo Legion Slim 5 AI Powered AMD Ryzen 7 Octa Core 7840HS - (16 GB/512 GB SSD/Windows 11 Hom...',
  'Product price': 'SGD 1827.04',
  'Number of ratings': 'Number of ratings not available',
  'Number of reviews': 'Number of reviews not available'},
 {'Product name': 'Lenovo Legion 5 Pro AMD Ryzen 7 Octa Core 6800H - (32 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVIDIA...',
  'Product price': 'SGD 1999.84',
  'Number of ratings': '28 Ratings\xa0',
  'Number of reviews': '\xa02 Reviews',
  'Review Summary': ['Review not available', 'Good choice'],
  'Review Detail': ['Review detail not available',
   "Awesome laptop with a few niggles. The display is crisp and vibrant, the performance i

In [16]:
df=pd.DataFrame(data)

In [20]:
df.to_csv("Product_data.csv")